In [1]:
import pandas as pd

import tweepy
import praw
import config

from bots.twitter import TwitterBot
from bots.reddit import RedditBot

2023-03-18 12:02:32,160 - INFO     | config     | Loading environment variables
2023-03-18 12:02:32,162 - WARNING  | config     | Directory raw_data already exists


## Twitter

In [2]:
twitter_bot = TwitterBot(bearer=config.TWTR_BEARER_TOKEN,
                         api=config.TWTR_API,
                         api_secret=config.TWTR_API_SECRET,
                         access=config.TWTR_ACCESS_TOKEN,
                         access_secret=config.TWTR_ACCESS_TOKEN_SECRET)

In [32]:
#query = 'entity:"Automotive" -is:retweet lang:en'
#context: brand | entity: products
query = 'context:46.783337567798169600 -is:retweet lang:en'

In [33]:
tweets = twitter_bot.get_recent_tweets(query=query1, limit=5000)

In [34]:
tweets_df = pd.DataFrame.from_dict(tweets)
tweets_df

,id,created_at,text,entities
0,1637050493657841664,2023-03-18 11:17:07+00:00,@smcglobal Honeywell\n#MoneywiseBeWise #SMC #S...,"{Personal finance, Honeywell, Stocks & indices..."
1,1637050492697317377,2023-03-18 11:17:07+00:00,A photocopy or xerox service is a common offer...,"{B2B, Products - B2B, Xerox, Services - B2B}"
2,1637050335301648385,2023-03-18 11:16:29+00:00,(US)PowerColor Red Devil AMD Radeon RX 7900 XT...,"{B2B, Radeon, Products - B2B, AMD}"
3,1637049850150748160,2023-03-18 11:14:33+00:00,@talkSPORT Rank Xerox,"{B2B, Products - B2B, Xerox, Services - B2B}"
4,1637049653928800256,2023-03-18 11:13:47+00:00,@smcglobal Ans. Honeywell\n\n#MoneywiseBeWise...,"{Personal finance, Honeywell, Stocks & indices..."
...,...,...,...,...
4995,1636336700132585475,2023-03-16 12:00:45+00:00,The full wireless package. 🎁 We’re proud to be...,"{B2B, Juniper Networks, Products - B2B, Servic..."
4996,1636336656172175364,2023-03-16 12:00:35+00:00,@twilitdreaming I'm doing an AMD build with th...,"{B2B, Products - B2B, AMD}"
4997,1636336585582198784,2023-03-16 12:00:18+00:00,Check out our latest blog article: why does m...,"{Honeywell, Blogging, Writing, B2B, Products -..."
4998,1636336535048994821,2023-03-16 12:00:06+00:00,"Amazon [$1,199.99]: Sapphire Nitro+ AMD Radeon...","{Gaming Business, AMD, B2B, Radeon, Products -..."


In [36]:
tweets_df.to_parquet(f'{config.DATA_DIR}/testing_cleaning.parquet', index=False)

## Data Cleaning

In [65]:
import re

In [66]:
df = pd.read_parquet(f'{config.DATA_DIR}/testing_cleaning.parquet')
df['created_at'] = pd.to_datetime(df['created_at'].dt.strftime('%Y-%m-%d'))

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          5000 non-null   int64         
 1   created_at  5000 non-null   datetime64[ns]
 2   text        5000 non-null   object        
 3   entities    5000 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 156.4+ KB


In [79]:
df.head()

,id,created_at,text,entities
0,1637050493657841664,2023-03-18,Honeywell\n#MoneywiseBeWise #SMC #SMCGlobal #...,"[Personal finance, Honeywell, Stocks & indices..."
1,1637050492697317377,2023-03-18,A photocopy or xerox service is a common offer...,"[B2B, Products - B2B, Xerox, Services - B2B]"
2,1637050335301648385,2023-03-18,(US)PowerColor Red Devil AMD Radeon RX 7900 XT...,"[B2B, Radeon, Products - B2B, AMD]"
3,1637049850150748160,2023-03-18,Rank Xerox,"[B2B, Products - B2B, Xerox, Services - B2B]"
4,1637049653928800256,2023-03-18,Ans. Honeywell\n\n#MoneywiseBeWise #SMC #SMC...,"[Personal finance, Honeywell, Stocks & indices..."


In [77]:
def text_cleaner(text):
    text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text)
    text = re.sub("@[A-Za-z0-9_]+","", text)
    return text

In [78]:
df['text'] = df['text'].map(lambda x: text_cleaner(x))

## Reddit

In [ ]:
reddit_bot = RedditBot(client_id=config.REDDIT_APP_ID,
                       client_secret=config.REDDIT_SECRET,
                       redirect_url=config.REDDIT_REDIRECT_URL,
                       user_agent=config.REDDIT_USER_AGENT)

In [ ]:
d = {}
for sub in reddit_bot.client.subreddit('MachineLearning').top(time_filter='all', limit=50):
    d['id'] = sub.id
    d['']

In [ ]:
t = []
for post in reddit_bot.client.subreddit('TalesFromTheCustomer').top(time_filter='all', limit=50):
    d = {}
    d['id'] = post.id
    d['text'] = post.title + '\n' + post.selftext
    t.append(d)

In [ ]:
comments = []
for post in t:
    post_id = post['id']
    
    submission = reddit_bot.client.submission(post_id)
    
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        comments.append(comment)
    break